In [1]:
import os
try:
    %run setup_paths
except:
    %run notebooks/setup_paths
    
print("current dir: ", os.getcwd())

current dir:  c:\Projects\scmsim


In [1]:
import os
#os.environ["OPENAI_API_KEY"] = "" ## PUT api key here / or better set in env outside

In [2]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under the silver moon, a gentle unicorn named Lila soared above the sleepy forest, sprinkling dreams of magic and happiness to all the children below.


In [3]:

PROMPT_OLD_DEPRICATED = """
You are an expert causal inference analyst using MovieLens data and domain knowledge.

Given two movie titles:

- Title A: "{title_A}"
- Title B: "{title_B}"

Estimate the **causal effect** of watching Title A on watching Title B, on a scale from -10 to 10, where:

-10 means watching Title A strongly causes someone NOT to watch Title B.
0 means watching Title A has no causal effect on watching Title B.
+10 means watching Title A strongly causes someone to watch Title B.

Important:

- Return a causal effect estimate, not a mere correlation.
- Consider and mention confounders like user preferences, release times, genre similarity, and franchise connections.
- Provide a concise explanation for the estimate.
- Format your response exactly as JSON with these fields:

{{
  "title_A": "{title_A}",
  "title_B": "{title_B}",
  "causal_effect": <integer from -10 to 10>,
  "explanation": "<brief text explaining the reasoning>"
}}

---

Example input:

Title A: "Star Wars (1977)"
Title B: "Empire Strikes Back, The (1980)"

Example output:

{{
  "title_A": "Star Wars (1977)",
  "title_B": "Empire Strikes Back, The (1980)",
  "causal_effect": 9,
  "explanation": "Empire Strikes Back is a direct sequel to Star Wars and watching the original strongly encourages watching the sequel, even after controlling for preferences and genre."
}}
"""

In [4]:
PROMPT = """
You are a causal inference expert analyzing movie-watching behavior using MovieLens-style data.

You are given a sequence of (A, B) movie title pairs. For each pair:

- Estimate the **causal effect** of watching movie A on watching movie B.
- Return a score from **–10 to 10**, where:
    - –10 means watching A *strongly causes* someone NOT to watch B
    - 0 means no causal effect
    - +10 means watching A *strongly causes* someone to watch B

This is **not correlation**. Estimate the **causal effect**, accounting for:
- Confounders like genre preference, popularity, release time, viewer behavior
- Whether B can stand alone (i.e., viewers may watch B without A)
- Whether A might disappoint and discourage further viewing

Some movies are **loose sequels or spiritual successors** — in those cases, B might be watched independently, but **watching A can still causally reduce** interest in B.

---

Return a JSON list where each entry has the following structure:

{
  "title_A": "<Movie A>",
  "title_B": "<Movie B>",
  "causal_effect": <integer from -10 to 10>,
  "explanation": "<brief justification of the score, including narrative dependency, reception, and genre factors>"
}

Only output valid JSON — no commentary or text outside the JSON block.

---
"""


In [5]:
from openai import OpenAI
client = OpenAI()

def get_movies_causal_score(title_pairs, prompt=PROMPT, model="gpt-4.1"):
    history = ([
        {"role":"user", "content":PROMPT}
    ] + [
        {"role":"user", "content":f"A: '{titleA}'; B: '{titleB}'"}    
        for titleA, titleB in title_pairs
    ])
    #print history
    response = client.responses.create(
        model="gpt-4.1",
        input=history
    )
    return response.output_text.strip()



In [6]:
example = [("The Hobbit","Return of the King"), ("Kill Bill", "Pulp Fiction")]
get_movies_causal_score(example)

'[\n  {\n    "title_A": "The Hobbit",\n    "title_B": "Return of the King",\n    "causal_effect": 2,\n    "explanation": "\'The Hobbit\' is a prequel to \'The Lord of the Rings\' trilogy, with overlapping lore and universe. Watching \'The Hobbit\' may slightly increase interest in \'Return of the King\' through established worldbuilding, but \'Return of the King\' stands strong on its own and is more popular, often viewed independently. Genre fans are likely to watch both regardless, and negative reception of \'The Hobbit\' may even modestly dampen enthusiasm for continuing. Overall, minor positive causal effect."\n  },\n  {\n    "title_A": "Kill Bill",\n    "title_B": "Pulp Fiction",\n    "causal_effect": 0,\n    "explanation": "Although both are Quentin Tarantino films and share stylistic elements and some cast overlap, there is no direct narrative connection. Fans of one may also like the other due to director loyalty or genre preference, but this is a confounder (genre/director tas

In [7]:
import pandas as pd
movies = pd.read_csv("MovieLensPairsSateShuf.csv")
movies["pid"] = (movies["treatment_idx"].astype(str) + ":" + movies["resp_idx"].astype(str))

In [8]:
movies

,idx,treatment_idx,resp_idx,treatment_title,resp_title,sate,pid
0,0,519,678,"Treasure of the Sierra Madre, The (1948)",Volcano (1997),0.003874,519:678
1,1,181,405,Return of the Jedi (1983),Mission: Impossible (1996),-0.525902,181:405
2,2,173,451,"Princess Bride, The (1987)",Grease (1978),0.966443,173:451
3,3,821,228,Mrs. Winterbourne (1996),Star Trek: The Wrath of Khan (1982),0.054022,821:228
4,4,357,405,One Flew Over the Cuckoo's Nest (1975),Mission: Impossible (1996),-0.458904,357:405
...,...,...,...,...,...,...,...
888301,888301,332,45,Kiss the Girls (1997),Eat Drink Man Woman (1994),0.109005,332:45
888302,888302,775,480,Something to Talk About (1995),North by Northwest (1959),0.109000,775:480
888303,888303,609,745,Father of the Bride (1950),"Ruling Class, The (1972)",0.109000,609:745
888304,888304,838,534,In the Line of Duty 2 (1987),Traveller (1997),0.109002,838:534


In [9]:
def get_pair_id(item):
    return str(item["treatment_idx"])+":"+str(item["resp_idx"])
    
def load_results(path):
    results = []
    with open(path, "rt") as rf:
        nmismatch = 0
        num_nonzero_effect =0
        for line in rf:    
            item = json.loads(line)
            if (item["treatment_title"]!=item["title_A"] or item["resp_title"]!=item["title_B"]):
                nmismatch += 1
                continue
                raise Exception("mismatch")            
            results.append(item)
            num_nonzero_effect += (item["causal_effect"] != 0)
        print(f"num-mismatch: {nmismatch}; num-non-zero-effect: {num_nonzero_effect}")
        return results

In [10]:
import json
from datetime import datetime

RESULTS_PATH = "products/MoviesCausalGPT.txt"
visited = set()

if os.path.isfile(RESULTS_PATH):
    prev_results = load_results(RESULTS_PATH)
    visited = set([get_pair_id(x) for x in prev_results])
    
print("visited:", len(visited))

with open(RESULTS_PATH, "a", encoding="utf-8") as outf:
    batch_size = 25
    df = movies        
    itr = 0
    effects = []
    while True:
        df = df[~(df["pid"].isin(visited))]
        if (len(df) == 0):
            break
            
        curr = df[0:batch_size]
        curr_list = curr.to_dict(orient='records')
        
        pairs = [(x["treatment_title"], x["resp_title"]) for x in curr_list]
        gpt_response = json.loads(get_movies_causal_score(pairs))

        
        for query, response in zip(curr_list, gpt_response):
            pid = get_pair_id(query)            
            res = {}
            res.update(query)
            res.update(response)
            res["explanation"] = res["explanation"].replace("\n","").replace("\r","")
            effects.append(res["causal_effect"])
            if "pid" in res:
                del res["pid"]
            print(json.dumps(res), file=outf)
            outf.flush()
            visited.add(pid)

        if (itr % 10 < 10):
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"[{itr}]: {timestamp} - remaining:{len(df)}; visited:{len(visited)}; min_effect:{min(effects)}; max_effect:{max(effects)};")
            effects = []
        itr += 1

    

num-mismatch: 25; num-non-zero-effect: 671
visited: 4905
[0]: 07:42:34 - remaining:883401; visited:4930; min_effect:0; max_effect:0;
[1]: 07:42:52 - remaining:883376; visited:4955; min_effect:0; max_effect:2;
[2]: 07:43:10 - remaining:883351; visited:4980; min_effect:-2; max_effect:2;
[3]: 07:43:30 - remaining:883326; visited:5005; min_effect:0; max_effect:2;
[4]: 07:44:25 - remaining:883301; visited:5030; min_effect:0; max_effect:0;
[5]: 07:44:54 - remaining:883276; visited:5055; min_effect:0; max_effect:1;
[6]: 07:44:56 - remaining:883251; visited:5056; min_effect:0; max_effect:0;
[7]: 07:45:25 - remaining:883250; visited:5081; min_effect:0; max_effect:0;
[8]: 07:45:54 - remaining:883225; visited:5106; min_effect:0; max_effect:3;
[9]: 07:46:30 - remaining:883200; visited:5131; min_effect:0; max_effect:0;
[10]: 07:47:22 - remaining:883175; visited:5156; min_effect:0; max_effect:1;
[11]: 07:47:50 - remaining:883150; visited:5181; min_effect:0; max_effect:2;
[12]: 07:48:11 - remaining:8

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [48]:
#response[0]["explanation"]

'There is no narrative, genre, or temporal connection between the classic adventure drama and the 1990s disaster film. Watching one does not influence the likelihood of watching the other beyond general movie-watching propensity, which is not causal for these specific titles.'

In [11]:
#gpt_response

In [12]:
#PROMPT

In [14]:
results  = load_results(RESULTS_PATH)
results[0]

num-mismatch: 43; num-non-zero-effect: 1244


{'idx': 0,
 'treatment_idx': 519,
 'resp_idx': 678,
 'treatment_title': 'Treasure of the Sierra Madre, The (1948)',
 'resp_title': 'Volcano (1997)',
 'sate': 0.0038741753,
 'title_A': 'Treasure of the Sierra Madre, The (1948)',
 'title_B': 'Volcano (1997)',
 'causal_effect': 0,
 'explanation': "There is no narrative or genre connection between a classic adventure/drama from 1948 and a 1990s disaster action film. Watching 'Treasure of the Sierra Madre' is highly unlikely to causally influence interest in or avoidance of 'Volcano'; their audiences and appeal are independent."}

In [20]:
cols = ["idx", "treatment_idx", "resp_idx", "treatment_title", "resp_title", "sate", "causal_effect", "explanation"]
results_df = pd.DataFrame([{ k:v for k,v in x.items() if k in cols} for x in results])

In [21]:
results_df

,idx,treatment_idx,resp_idx,treatment_title,resp_title,sate,causal_effect,explanation
0,0,519,678,"Treasure of the Sierra Madre, The (1948)",Volcano (1997),0.003874,0,There is no narrative or genre connection betw...
1,1,181,405,Return of the Jedi (1983),Mission: Impossible (1996),-0.525902,0,"While both are blockbuster franchises, there i..."
2,2,173,451,"Princess Bride, The (1987)",Grease (1978),0.966443,0,'The Princess Bride' (fantasy/romance/comedy) ...
3,3,821,228,Mrs. Winterbourne (1996),Star Trek: The Wrath of Khan (1982),0.054022,0,These films are disconnected by genre (romanti...
4,4,357,405,One Flew Over the Cuckoo's Nest (1975),Mission: Impossible (1996),-0.458904,0,"Although both are well-known films, their genr..."
...,...,...,...,...,...,...,...,...
9200,9218,809,920,Rising Sun (1993),Two Bits (1995),0.642178,0,"'Rising Sun' is a crime thriller, 'Two Bits' i..."
9201,9219,124,833,Lone Star (1996),Bulletproof (1996),0.642089,0,"'Lone Star' is a slow-burn mystery drama, 'Bul..."
9202,9220,353,59,Deep Rising (1998),Three Colors: Red (1994),-0.108727,0,"'Deep Rising' is a monster-action film, while ..."
9203,9221,84,47,Robert A. Heinlein's The Puppet Masters (1994),Ed Wood (1994),0.045812,0,"Although both films are from 1994, 'The Puppet..."


In [22]:
results_df.to_csv("products/MoviesCausalGPT.csv", index=False)

In [23]:
(results_df["causal_effect"] != 0).sum() / len(results_df)

0.13514394350896253